In [1]:
import mysql.connector as connector

In [2]:
# Establish connection between Python and MySQL database via MySQL Connector/Python API
connection=connector.connect(
                             user="root",
                             password="",
                            )

In [3]:
# Create cursor object to communicate with entire MySQL database
cursor = connection.cursor()

Once the connection is established and you have a cursor object, you can select the database `little_lemon` using the code below and work with the respective table to complete the required tasks.  

In [4]:
# Set the little_lemon database for use 
cursor.execute("use little_lemon_db")

# Confirm the datbase in use
connection.database

'little_lemon_db'

# Exercise 2: Little Lemon analysis and sales report
## Task 1:

Complete the following steps to establish a connection pool:

To create a connection pool, import the MySQLConnectionPool class from MySQL Connector/Python.

To find the information on the error, import the Error class from MySQL Connector/Python.

Define your database configurations as a Python dictionary object called dbconfig.

Establish a connection pool [pool_name = “pool_b”] with two connections. 

Implement error handling using a try-except block in case the connection fails. 

In [10]:
# Import MySQLConnectionPool class
from mysql.connector.pooling import MySQLConnectionPool

# Import Error class
from mysql.connector import Error

# Define database configurations
dbconfig = {
    "database":"little_lemon_db",
    "user" : "root",
    "password" : ""
}

# Create a pool named "pool_b" with two connections.
# try-except block
try:
    pool = MySQLConnectionPool(pool_name = "pool_b",pool_size = 2,**dbconfig)
    print("The connection pool is created with the name: ",pool.pool_name)
    print("The pool size is:",pool.pool_size)

except Error as er:
    print("Error code:", er.errno)
    print("Error message:", er.msg)


The connection pool is created with the name:  pool_b
The pool size is: 2


## Task 2:

Three guests are trying to book dinner slots simultaneously. Get their connections from pool_b and insert the following data in the Bookings table. (TIP: You need to add a connection to connect the third guest.)


Guest 1:

Table Number: 8

First Name: Anees

Last Name: Java

Booking Time: 18:00

EmployeeID: 6


Guest 2:

Table Number: 5

First Name: Bald

Last Name: Vin

Booking Time: 19:00

EmployeeID: 6


Guest 3:

Table Number: 12

First Name: Jay

Last Name: Kon

Booking Time: 19:30 

EmployeeID: 6


Return all the connections back to the pool. (TIP: The pool size is 2. However, you have three connected users. You can only return two connections. Returning a third connection raises a PoolError. Use try-except to print the error message)  

In [ ]:
#Connect the first guest
connection1 = pool.get_connection()
cursor1 = connection1.cursor()
booking1 = """
INSERT INTO Bookings
(TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID)
VALUES
(8, 'Anes', 'Java', '18:00:00', 6)
"""
cursor1.execute(booking1)
connection1.commit()
print("A new booking booking has been added into the 'booking' table.")

#Connect the second guest
connection2 = pool.get_connection()
cursor2 = connection2.cursor()
booking2 = """
INSERT INTO Bookings
(TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID)
VALUES
(5, 'Bald', 'Vin', '19:00:00', 6)
"""
cursor2.execute(booking2)
connection2.commit()
print("A new booking booking has been added into the 'booking' table.")

# Adding a new connection to connect the third user.
try:
    connection3 = pool.get_connection()
    print("The guest is connected")
except:
    print("Adding new connection in the pool.")
        
    # Create a connection
    connection = connector.connect(user="meta",password="password")
    # Add the connection into the pool
    pool.add_connection(cnx=connection)
    print("A new connection is added in the pool.\n")
        
    connection3 = pool.get_connection()
    print("'connection3' has been added into the pool.")
    
# Connect the third guest 
cursor3 = connection3.cursor()
booking3 = """INSERT INTO Bookings 
(TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID)
VALUES
(12, 'Jay','Kon','19:30:00',6);"""
cursor3.execute(booking3)
connection3.commit()
print("A new booking booking has been added into the 'booking' table.")

# You can only return two connections back to the pool as the pool_size=2.
#  Close all  connections and use try-except to print the pool error if the pool is already full.

from mysql.connector import Error
for connection in [connection1,connection2, connection3]:
    try:
        connection.close()
        print("Connection is returned to the pool")
    except Error as er:
        print("\nConnection can't be returned to the pool")
        print("Error message:", er.msg)

## Task 3:

Create a report containing the following information:

The name and EmployeeID of the Little Lemon manager.

The name and role of the employee who receives the highest salary.

The number of guests booked between 18:00 and 20:00.

The full name and the BookingId of each guest waiting to be seated with the receptionist in sorted order with respect to their BookingSlot.

In [22]:
# Get a connection from pool_a and create a cursor object to communicate with the database. 
print("Getting a connection from the pool.")
connection = pool.get_connection()

print("""Creating a cursor object to communicate with the database.""")
cursor = connection.cursor()
print("""The cursor object "cursor" is created.""")
#The name and EmployeeID of the Little Lemon manager.
cursor.execute("""SELECT 
Name, EmployeeID 
FROM Employees 
WHERE Role = 'Manager'""")
results=cursor.fetchall()
columns=cursor.column_names
print(columns)
for result in results:
    print(result)

#The name and role of the employee who receives the highest salary.
cursor.execute("""SELECT 
Name, Role 
FROM Employees 
ORDER BY Annual_Salary DESC
LIMIT 1;""")
results=cursor.fetchall()
columns=cursor.column_names
print(columns)
for result in results:
    print(result)

#The number of guests booked between 18:00 and 20:00
cursor.execute("""SELECT 
COUNT(BookingID) AS n_Booking_18_20
FROM Bookings 
WHERE BookingSlot BETWEEN '18:00:00' AND '20:00:00' ;""")
results=cursor.fetchall()
columns=cursor.column_names
print(columns)
for result in results:
    print(result)

#The full name and the BookingId of each guest waiting to be seated with the receptionist in sorted order with respect to their BookingSlot.
cursor.execute("""SELECT 
Bookings.BookingID AS n_Booking,
CONCAT(GuestFirstName, ' ', GuestLastName) AS GuestName,
Role AS Employee
FROM Bookings 
LEFT JOIN Employees
ON Employees.EmployeeID=Bookings.EmployeeID
WHERE Role = 'Receptionist'
ORDER BY BookingSlot DESC ;""")
print("The following guests are waiting to be seated:")
results=cursor.fetchall()
columns=cursor.column_names
print(columns)
for result in results:
    print(result)


Getting a connection from the pool.
Creating a cursor object to communicate with the database.
The cursor object "cursor" is created.
('Name', 'EmployeeID')
('Mario Gollini', 1)
('Name', 'Role')
('Mario Gollini', 'Manager')
('n_Booking_18_20',)
(8,)
The following guests are waiting to be seated:
('n_Booking', 'GuestName', 'Employee')
(8, 'Bald Vin', 'Receptionist')
(10, 'Bald Vin', 'Receptionist')
(7, 'Anes Java', 'Receptionist')
(9, 'Anes Java', 'Receptionist')


## Task 4:

Create a stored procedure called BasicSalesReport that returns the following statistics. 

Total sales

Average sale

Minimum bill paid

Maximum bill paid

In [8]:
# Create a stored procedure named BasicSalesReport.
cursor.execute("""DROP PROCEDURE IF EXISTS BasicSalesReport;""")

stored_procedure_query = """ CREATE PROCEDURE BasicSalesReport()
BEGIN

SELECT
SUM(BillAmount) AS TotalSales,
AVG(BillAmount) AS AverageSales,
MIN(BillAmount) AS MinimumBillPaid,
MAX(BillAmount) AS MAXimumBillPaid
FROM Orders;

END
"""

#Execute the query.
cursor.execute(stored_procedure_query)
#Call The procedure
cursor.callproc("BasicSalesReport")

#Retrieve the record in dataset
results = next(cursor.stored_results())
results = results.fetchall()

# Retrieve column names using list comprehension in a for loop 
for column_id in cursor.stored_results():
    columns = [column[0] for column in column_id.description]


#Print report
print("Today's report")
for result in results:
    print(columns[0], ":", result[0])
    print(columns[1], ":", result[1])
    print(columns[2], ":", result[2])
    print(columns[3], ":", result[3])

Today's report
TotalSales : 243
AverageSales : 48.6000
MinimumBillPaid : 37
MAXimumBillPaid : 86


## Task 5:       

Little Lemon needs to display the next three upcoming bookings from the Bookings table on the kitchen screen to notify their chefs that the orders are due. 

Get a connection from the pool.

Create a buffered cursor.

Combine the data from the Bookings and the Employee tables, sort the retrieved records in ascending order, and display the information of the first three guests. 

Return the connection back to the pool.

The output should be as follows:

[BookingSlot]

[Guest_name]

[Assigned to: Employee Name [Employee Role]]

In [12]:
connection = pool.get_connection()
cursor = connection.cursor(buffered= True)

query = """
SELECT
Bookings.BookingSlot,
CONCAT(GuestFirstName, ' ', GuestLastName) AS GuestName,
Employees.Name AS EmployeeName,
Employees.Role AS EmployeeRole
FROM Bookings
INNER JOIN Employees
ON Bookings.EmployeeID = Employees.EmployeeID
ORDER BY Bookings.BookingSlot ASC;
"""
cursor.execute(query)
results = cursor.fetchmany(size=3)

for column_id in cursor.stored_results():
    columns = [column[0] for column in column_id.description]

print("Today's report")
for result in results:
    print( "BookingSlot:", result[0])
    print( "GuestName:", result[1])
    print( "Assigned to:", result[2], "[{}]".format(result[3]))

connection.close()

Today's report
BookingSlot: 15:00:00
GuestName: Vanessa McCarthy
Assigned to: Giorgos Dioudis [Head Chef]
BookingSlot: 17:30:00
GuestName: Marcos Romero
Assigned to: Fatma Kaya [Assistant Chef]
BookingSlot: 18:30:00
GuestName: Hiroki Yamane
Assigned to: Adrian Gollini [Assistant Manager]
